In [13]:
import pandas as pd
import numpy as np
import sqlite3
import statsmodels.formula.api as smf

try: 
    import yfinance as yf
except:
    !pip install yfinance
    import yfinance as yf

#from regtabletotext import prettify_result

In [32]:
prices = (yf.download(tickers="AAPL", 
                      start="2000-01-01", 
                      end="2022-12-31")
  .reset_index()
  .assign(symbol = "AAPL")
  .rename(columns = {"Date": "date", 
                     "Open": "open", 
                     "High": "high",
                     "Low": "low",
                     "Close": "close", 
                     "Adj Close": "adjusted", 
                     "Volume": "volume"
                    })
)

[*********************100%%**********************]  1 of 1 completed


In [37]:
prices.head().round(3)

,date,open,high,low,close,adjusted,volume,symbol
0,2000-01-03,0.936,1.004,0.908,0.999,0.847,535796800,AAPL
1,2000-01-04,0.967,0.988,0.903,0.915,0.776,512377600,AAPL
2,2000-01-05,0.926,0.987,0.920,0.929,0.787,778321600,AAPL
3,2000-01-06,0.948,0.955,0.848,0.848,0.719,767972800,AAPL
4,2000-01-07,0.862,0.902,0.853,0.888,0.753,460734400,AAPL
